In [1]:
import numpy as np
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import torch.optim as optim
import time
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from collections import Counter
from sklearn.metrics import confusion_matrix
import pandas as pd
import itertools
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [ ]:
# !unzip '/content/drive/MyDrive/data/zipdata.zip' -d '/content/drive/MyDrive/data/image_data'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
transform = transforms.Compose([
    transforms.Resize(256),        # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),   # Crop the center 224x224 pixels
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    # transforms.Normalize(         # Normalize the image
    #     mean=[0.485, 0.456, 0.406],
    #     std=[0.229, 0.224, 0.225]
    # )
])

data_dir = '/content/drive/MyDrive/Colab Notebooks/cnn/image_data'
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [4]:
total_count=len(dataset)
train_count = int(0.7 * total_count) 
valid_count = total_count - train_count 
train_dataset, valid_dataset = torch.utils.data.random_split(dataset=dataset, lengths=[train_count, valid_count])
batch_size=16
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
valid_dataset_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False) 
dataloaders = {'train': train_dataset_loader, 'valid': valid_dataset_loader}

In [5]:
def train_model(model, dataloaders, criterion, optimizer,device, num_epochs=50):
    since = time.time()
    best_acc = 0
    model.to(device)

    val_acc_history = []
    train_acc_history = []

    test_acc_history = []
    train_losses = []
    valid_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

          
            for inputs, labels in dataloaders[phase]:
                #print(inputs.shape)
                inputs = inputs.to(device)
                labels = labels.to(device)

           
                optimizer.zero_grad()
            
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

               
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            
            time_elapsed = time.time() - since
   
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                valid_losses.append(epoch_loss)

            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)
            


    return model, val_acc_history, train_acc_history,valid_losses, train_losses


# define plot accuracy and loss
def plot_acc_loss(val_acc_history, train_acc_history,valid_losses, train_losses):
  val_acc=[]
  for i in val_acc_history:
    val_acc.append(i.cpu().data.numpy())
  train_acc=[]
  for i in train_acc_history:
    train_acc.append(i.cpu().data.numpy())
  test_acc=[]

  
  plt.subplot(1,2,1)
  plt.plot(range(len(val_acc)),val_acc,label='val_acc')
  plt.plot(range(len(train_acc)),train_acc,label='train_acc')
  plt.xlabel('epoch')
  plt.ylabel('acc')
  plt.legend()
  

  plt.subplot(1,2,2)
  plt.plot(range(len(valid_losses)),valid_losses,label='val_loss')
  plt.plot(range(len(train_losses)),train_losses,label='train_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.show()

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
from torchvision import models
import torch.nn as nn
model = models.resnet50(pretrained=True)  #Use the resnet18 model
num_ftrs =model .fc.in_features #Modify the number of model categories
model.fc = nn.Sequential(nn.Linear(num_ftrs, 24))

In [ ]:
model

In [ ]:
class resnet(nn.Module):
  def __init__(self):
      super(resnet,self).__init__()
      self.model =models.resnet50(pretrained=True)
      
      
      self.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 24),  #Modify the last layer of the network
                                   nn.LogSoftmax(dim=1))
      
  def forward(self, x):
      
      x = self.model.avgpool(x)
      x = self.fc(x)
      return x

In [ ]:
model=resnet()
model

In [ ]:
from torchvision import models
import torch.nn as nn
model = models.resnet50(pretrained=True)  #Use the resnet18 model
num_ftrs =model .fc.in_features #Modify the number of model categories
model.fc = nn.Sequential(nn.Dropout(0.9),nn.Linear(num_ftrs, 24))


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cost_fun = nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5) 
model = model.to(device)
model, val_acc_history, train_acc_history,valid_losses, train_losses= train_model(model, dataloaders, cost_fun, optimizer, device,num_epochs=100)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/99
----------
train Loss: 3.5733 Acc: 0.0514
valid Loss: 3.0463 Acc: 0.1451
Epoch 1/99
----------
train Loss: 3.3836 Acc: 0.0734
valid Loss: 2.9151 Acc: 0.2006
Epoch 2/99
----------
train Loss: 3.1786 Acc: 0.1010
valid Loss: 2.7984 Acc: 0.2199
Epoch 3/99
----------
train Loss: 3.0043 Acc: 0.1350
valid Loss: 2.7007 Acc: 0.2485
Epoch 4/99
----------
train Loss: 2.8429 Acc: 0.1873
valid Loss: 2.6003 Acc: 0.2687
Epoch 5/99
----------
train Loss: 2.7232 Acc: 0.2160
valid Loss: 2.5127 Acc: 0.2931
Epoch 6/99
----------
train Loss: 2.5942 Acc: 0.2562
valid Loss: 2.4480 Acc: 0.3188
Epoch 7/99
----------
train Loss: 2.4563 Acc: 0.2993
valid Loss: 2.3878 Acc: 0.3272
Epoch 8/99
----------
train Loss: 2.3052 Acc: 0.3443
valid Loss: 2.3379 Acc: 0.3385
Epoch 9/99
----------
train Loss: 2.1442 Acc: 0.3775
valid Loss: 2.2973 Acc: 0.3520
Epoch 10/99
----------
train Loss: 1.9855 Acc: 0.4316
valid Loss: 2.2592 Acc: 0.3667
Epoch 11/99
----------
train Loss: 1.8321 Acc: 0.4795
valid Loss: 2.2440 Ac